# Web Scraping con Python

Este script muestra algunas posibilidades de la biblioteca *Selenium* que nos permite obtener, automáticamente, datos de páginas web dinámicas, es decir que necesitan pulsar botones, etc, etc.

Selenium está pensado realmente para automatizar pruebas de entornos web, pero a nosotros nos servirá para nuestro propósito de hacer web scraping.

En particular, lo que vamos a hacer es arrancar google Chrome y manejarlo automáticamente desde Python.

Para ello necesitamos 3 cosas:

1.- Tener google Chrome

2.- Instalar la biblioteca selenium

3.- Tener un fichero controlador, un driver

Empezamos asegurarnos de que Selenium está instalado

In [2]:
modules = ["selenium","chromedriver_autoinstaller"]


import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
    print("Instalando módulos")
    for m in modules:
        # para el import quitamos [...] y ==...
        p = m.find("[")
        mi = m if p==-1 else m[:p]
        p = mi.find("==")
        mi = mi if p==-1 else mi[:p]
        torch_loader = importlib.util.find_spec(mi)
        if torch_loader is not None:
            print(m," encontrado")
        else:
            print(m," No encontrado, instalando...",end="")  
            try:        
                r = check_call([sys.executable, "-m", "pip", "install", "--user",  m])
                print("¡hecho!")
            except:
                print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

    print("¡Terminado!")

instala(modules)  

Instalando módulos
selenium  encontrado
chromedriver_autoinstaller  encontrado
¡Terminado!


Ahora abrimos el navegador

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless') # ensure GUI is off
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()


# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

In [4]:
url = 'https://www1.sedecatastro.gob.es/CYCBienInmueble/OVCBusqueda.aspx'
driver.get(url)



Simulamos un click en la página de aceptar cookies


In [5]:

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
#aceptar las cookies

try:
    cookies = driver.find_element(By.LINK_TEXT, "Aceptar cookies")
    cookies.click()
    print("Cookies aceptadas.")
except Exception as e:
    print("No se encontró el botón de aceptar cookies o ocurrió un error:", e)

# Hacer clic en la pestaña de coordenadas
try:
    coordenadas_tab = driver.find_element(By.LINK_TEXT, "COORDENADAS")
    coordenadas_tab.click()
    print("Se ha accedido a la pestaña de coordenadas.")
except Exception as e:
    print("No se encontró la pestaña de coordenadas o ocurrió un error:")
#coord.click()

Cookies aceptadas.
Se ha accedido a la pestaña de coordenadas.


datos de Coordenadas

In [6]:
#lat = driver.find_element_by_id("ctl00_Contenido_txtLatitud")
#lon = driver.find_element_by_id("ctl00_Contenido_txtLongitud")
lat = driver.find_element(By.ID, "ctl00_Contenido_txtLatitud")

lon = driver.find_element(By.ID, "ctl00_Contenido_txtLongitud")
latitud  = "28.2723368"
longitud = "-16.64268"
lat.send_keys(latitud)
lon.send_keys(longitud)

Pulsar Botón para acceder a los datos concretos

In [7]:
datos = driver.find_element(By.ID, "ctl00_Contenido_btnDatos")
datos.click()



In [7]:
id = "ctl00_Contenido_tblInmueble"
#div = driver.find_element("id")
labels = driver.find_elements(By.XPATH, "//label")
for i,l in enumerate(labels):
    print(i,l.text)

0 38026A035000010000EI    
1 
2 
3 Polígono 35 Parcela 1 PARQUE NACINAL DEL TEIDE
MONTE. LA OROTAVA (S.C. TENERIFE)
4 Rústico
5 Agrario
6 
7 3.704 m2
8 1965
9 Polígono 35 Parcela 1 PARQUE NACINAL DEL TEIDE
MONTE. LA OROTAVA (S.C. TENERIFE)
10 73.628.811 m2
11 
12 
13 
14 
15 
16 
17 
18 


Referencia catastral

In [8]:
id = "ctl00_Contenido_tblInmueble"
pos = [0,3,4,5,7,8]
div = driver.find_element(By.ID,id)
labels = div.find_elements(By.XPATH,"//label")
for i,label in enumerate(labels):
    if i in pos:
        print(label.text)

38026A035000010000EI    
Polígono 35 Parcela 1 PARQUE NACINAL DEL TEIDE
MONTE. LA OROTAVA (S.C. TENERIFE)
Rústico
Agrario
3.704 m2
1965


Todo Junto

In [9]:
import pandas as pd
file = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/catastro.csv"
df = pd.read_csv(file)
df

,_id,followers,location,screen_name,verified,long,lat
0,148076807,515,"19.447297,-70.678351",eugenio_sky,False,-3.685060,40.427170
1,557076972,1022,BCN,annetapedro,False,2.176402,41.405603
2,800430827947442176,20,"Valencia, España",Trastarteando,False,-0.293131,39.567629
3,1187834312,653,"Madrid, Spain",manukareol,False,-3.705050,40.438520
4,3153928585,99,"Málaga, Andalucía",PedroMendez_mlg,False,-4.407480,36.720087
5,18901057,1562,UK/Brasil/España,GuitarCafe,False,0.060693,40.052917
6,117884436,1502,Barcelona,Ldiazalonso,False,-3.707360,40.421380
7,761237429072388096,35,"Madrid, Comunidad de Madrid",fjdemi,False,-3.584300,40.360320
8,184959473,733,LA MANCHA y SEVILLA,DonaireAlberto,False,-3.930790,38.986850
9,2838312285,27,España,begonabotana,False,-8.410720,43.355270


In [10]:
from time import sleep
df = pd.read_csv(file)
coldf = len(df.columns)
url = 'https://www1.sedecatastro.gob.es/CYCBienInmueble/OVCBusqueda.aspx'
# mejor con una lista de lsitas, pero así se entiende mejor
pos = [0,3,4,5,7,8]
nombres = ["ref","loc","clase","uso","superficie","año"]
df[nombres] = None
filenum = 0
for row in df.iterrows():
    # los datos están en row[1], row[0] es el índice
    fila = row[1]
    driver.get(url)
    coord   = driver.find_element(By.LINK_TEXT,"COORDENADAS")
    coord.click()
    sleep(0.5)
    # coordenadas
    lat = driver.find_element(By.ID,"ctl00_Contenido_txtLatitud")
    lon = driver.find_element(By.ID,"ctl00_Contenido_txtLongitud")
    latitud  = str(fila["lat"])
    longitud = str(fila["long"])
    lat.send_keys(latitud)
    lon.send_keys(longitud)
    sleep(0.25)
    # pulsar el botón, esto puede tardar
    datos = driver.find_element(By.ID,"ctl00_Contenido_btnDatos")
    datos.click()

    sleep(1.5)
    id = "ctl00_Contenido_tblInmueble"
    pos = [0,3,4,5,7,8]
    div = driver.find_elements(By.ID,id)
    if len(div)==1:
        div = div[0]
        labels = div.find_elements(By.XPATH,"//label")
        for i,label in enumerate(labels):
            if i in pos:
                df.iloc[filenum,coldf+pos.index(i)] = label.text
    filenum+=1



In [11]:
df


,_id,followers,location,screen_name,verified,long,lat,ref,loc,clase,uso,superficie,año
0,148076807,515,"19.447297,-70.678351",eugenio_sky,False,-3.685060,40.427170,2055317VK4725E0001BL,CL LAGASCA 60\n28001 MADRID (MADRID),Urbano,Residencial,2.986 m2,1900
1,557076972,1022,BCN,annetapedro,False,2.176402,41.405603,1242504DF3814C0001BL,CL MALLORCA 425\n08013 BARCELONA (BARCELONA),Urbano,Comercial,4.077 m2,1993
2,800430827947442176,20,"Valencia, España",Trastarteando,False,-0.293131,39.567629,2634601YJ3823S0001XX,AV MAR 25 Bl:A\n46137 LA POBLA DE FARNALS (VAL...,Urbano,Deportivo,5.754 m2,1978
3,1187834312,653,"Madrid, Spain",manukareol,False,-3.705050,40.438520,0167607VK4706G0001EM,CL BRAVO MURILLO 47 TEATROS DEL CANAL\n28015 M...,Urbano,Espectáculos,38.382 m2,2008
4,3153928585,99,"Málaga, Andalucía",PedroMendez_mlg,False,-4.407480,36.720087,4449101UF7644N0001SA,PS REDING 22\n29016 MALAGA (MÁLAGA),Urbano,"Ocio,Hostelería",24.510 m2,1936
5,18901057,1562,UK/Brasil/España,GuitarCafe,False,0.060693,40.052917,9480003BE4398S0001WA,AV CASTELLON 19\n12560 BENICASIM / BENICASSIM ...,Urbano,Cultural,798 m2,1940
6,117884436,1502,Barcelona,Ldiazalonso,False,-3.707360,40.421380,0149302VK4704G0001QS,CL GRAN VIA 54\n28013 MADRID (MADRID),Urbano,Oficinas,9.315 m2,1929
7,761237429072388096,35,"Madrid, Comunidad de Madrid",fjdemi,False,-3.584300,40.360320,0480404VK5608A0001WP,AV CERRO MILANO 537 U.E.5\n28031 MADRID (MADRID),Urbano,Residencial,247 m2,2019
8,184959473,733,LA MANCHA y SEVILLA,DonaireAlberto,False,-3.930790,38.986850,9559001VJ1195N0001IQ,CL PRADO 5\n13002 CIUDAD REAL (CIUDAD REAL),Urbano,Religioso,2.968 m2,1500
9,2838312285,27,España,begonabotana,False,-8.410720,43.355270,7807017NJ4070N0001TY,CL SAN VICENTE 16\n15007 A CORUÑA (A CORUÑA),Urbano,Residencial,494 m2,1958


### Más pruebas

Texto de la página

In [12]:
html = driver.find_element(By.XPATH,"/html")
print(html.text)

Formulario master
Castellano
ICONO CORREO ELECTRÓNICO
CONTÁCTENOS
Icono página de inicio
Consulta y certificación de Bien Inmueble
Volver
CARTOGRAFÍA
CONSULTA DESCRIPTIVA Y GRÁFICA
EXPEDIENTES ABIERTOS
IMPRIMIR DATOS
VISOR 3D
DATOS DESCRIPTIVOS DEL INMUEBLE
Referencia catastral
6009307BR0660N0001GO    
Localización
CL JUAN GUTIERREZ MONTEVER 20 LA RESTINGA
38914 EL PINAR DE EL HIERRO (RESTINGA LA) (S.C. TENERIFE)
Clase
Urbano
Uso principal
Residencial
Superficie construida
102 m2
Año construcción
1995
PARCELA CATASTRAL

Parcela construida sin división horizontal
Localización
CL JUAN GUTIERREZ MONTEVER 20 LA RESTINGA
EL PINAR DE EL HIERRO (RESTINGA LA) (S.C. TENERIFE)
Superficie gráfica
48 m2
CONSTRUCCIÓN
Uso principal Escalera Planta Puerta Superficie m2 Tipo Reforma Fecha Reforma
VIVIENDA 00 01 48
VIVIENDA 01 48
ALMACEN 02 01 6
¿Cómo se pueden obtener datos protegidos (titularidad y valor catastral) de los inmuebles y certificados telemáticos de los mismos?








Normativa regulador

Caminos absolutos

In [13]:
head = driver.find_element(By.XPATH,"/html/head")
body = driver.find_element(By.XPATH,"/html/body")
html2 = body.find_element(By.XPATH,"/html")

In [14]:
print(body.text)

Formulario master
Castellano
ICONO CORREO ELECTRÓNICO
CONTÁCTENOS
Icono página de inicio
Consulta y certificación de Bien Inmueble
Volver
CARTOGRAFÍA
CONSULTA DESCRIPTIVA Y GRÁFICA
EXPEDIENTES ABIERTOS
IMPRIMIR DATOS
VISOR 3D
DATOS DESCRIPTIVOS DEL INMUEBLE
Referencia catastral
6009307BR0660N0001GO    
Localización
CL JUAN GUTIERREZ MONTEVER 20 LA RESTINGA
38914 EL PINAR DE EL HIERRO (RESTINGA LA) (S.C. TENERIFE)
Clase
Urbano
Uso principal
Residencial
Superficie construida
102 m2
Año construcción
1995
PARCELA CATASTRAL

Parcela construida sin división horizontal
Localización
CL JUAN GUTIERREZ MONTEVER 20 LA RESTINGA
EL PINAR DE EL HIERRO (RESTINGA LA) (S.C. TENERIFE)
Superficie gráfica
48 m2
CONSTRUCCIÓN
Uso principal Escalera Planta Puerta Superficie m2 Tipo Reforma Fecha Reforma
VIVIENDA 00 01 48
VIVIENDA 01 48
ALMACEN 02 01 6
¿Cómo se pueden obtener datos protegidos (titularidad y valor catastral) de los inmuebles y certificados telemáticos de los mismos?








Normativa regulador

Hijos de un elemento

In [15]:
hijos = driver.find_elements(By.XPATH,"/html/body/*")
for element in hijos:
  print(element.tag_name)

div
div
form
div
script
a
script
link
script
script
script


Camino relativo

In [ ]:
divs = driver.find_elements(By.XPATH,"/html/body/*/div")
print(len(divs))

In [ ]:
divs = body.find_elements(By.XPATH,"./*/div")
print(len(divs))

Saltar pasos intermedios

In [ ]:
divs = driver.find_elements(By.XPATH,"/html/body//div")
print(len(divs))

In [ ]:
labels = driver.find_elements(By.XPATH,"//label")
print(len(labels))

Insertar texto

In [ ]:
lat = driver.find_element(By.ID,"ctl00_Contenido_txtLatitud")
lon = driver.find_element(By.ID,"ctl00_Contenido_txtLongitud")
latitud  = "28.2723368"
longitud = "-16.64268"
lat.send_keys(latitud)
lon.send_keys(longitud)



In [ ]:
datos = driver.find_element(By.ID,"ctl00_Contenido_btnDatos")
datos.click()

In [ ]:
id = "ctl00_Contenido_tblInmueble"
div = driver.find_element(By.ID,id)
label = div.find_element(By.XPATH,"//label")
print(label.text)

Otra forma, por texto

In [ ]:
xpath = "//*[./span/text()='Referencia catastral']//label"
etiqs = driver.find_element(By.XPATH,xpath)
print(etiqs.text)

Algunos ejemplos más....

In [ ]:
clase = driver.find_elements(By.XPATH,"(//label)[position()=3]")
print(clase[0].text)

etiqs = driver.find_elements(By.XPATH,"//label")
print(etiqs[2].text)

ulti = driver.find_elements(By.XPATH,"(//label)[last()]")
print(ulti[0].text)


Un pequeño ejemplo adicional, información metereológica en Madrid

In [ ]:
url2 = 'http://www.aemet.es/es/eltiempo/prediccion/municipios/madrid-id28079'
driver.get(url2)


In [ ]:
th = driver.find_elements(By.XPATH,"/html/body//*/th")
print(len(th))
th = driver.find_elements(By.XPATH,"/html/body//*/th[@class='borde_izq_dcha_estado_cielo no_wrap']")
print(len(th))
for e in th:
    print(e.text)

In [16]:
driver.close()

Por Rafael Caballero. Del libro "Big data con Python". Gracias a José Ramón Guerra por las actualizaciones